In [2]:
!pwd

/content


In [3]:
import os
os.chdir('/content/drive/MyDrive/Projects/fake news detection')

In [4]:
"""
Fake news detection
LSTM model
"""
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from collections import Counter
import os
import getEmbeddings
import matplotlib.pyplot as plt

In [5]:
top_words = 5000
epoch_num = 15
batch_size = 64

In [6]:
# Read the text data
if not os.path.isfile('./xtr_shuffled.npy') or \
    not os.path.isfile('./xte_shuffled.npy') or \
    not os.path.isfile('./ytr_shuffled.npy') or \
    not os.path.isfile('./yte_shuffled.npy'):
    getEmbeddings.clean_data()

In [7]:
xtr = np.load('./xtr_shuffled.npy', allow_pickle= True)
xte = np.load('./xte_shuffled.npy', allow_pickle= True)
y_train = np.load('./ytr_shuffled.npy', allow_pickle= True)
y_test = np.load('./yte_shuffled.npy', allow_pickle= True)

In [8]:
cnt = Counter()
x_train = []
for x in xtr:
    x_train.append(x.split())
    for word in x_train[-1]:
        cnt[word] += 1  

In [9]:
# Storing most common words
most_common = cnt.most_common(top_words + 1)
word_bank = {}
id_num = 1
for word, freq in most_common:
    word_bank[word] = id_num
    id_num += 1

In [10]:
# Encode the sentences
for news in x_train:
    i = 0
    while i < len(news):
        if news[i] in word_bank:
            news[i] = word_bank[news[i]]
            i += 1
        else:
            del news[i]

In [11]:
y_train = list(y_train)
y_test = list(y_test)

In [12]:
# Delete the short news
i = 0
while i < len(x_train):
    if len(x_train[i]) > 10:
        i += 1
    else:
        del x_train[i]
        del y_train[i]

In [13]:
# Generating test data
x_test = []
for x in xte:
    x_test.append(x.split())

In [14]:
# Encode the sentences
for news in x_test:
    i = 0
    while i < len(news):
        if news[i] in word_bank:
            news[i] = word_bank[news[i]]
            i += 1
        else:
            del news[i]

In [15]:
# Truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [16]:
# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [17]:
# Create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+2, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch_num, batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160064    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,365
Trainable params: 213,365
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
251/251 [==============================] - 17s 35ms/step - loss: 0.3133 - accuracy: 0.8700 - val_loss: 0.4923 - val_accuracy: 0.8139
Epoch 2/15
251/251 [==============================] - 8s 31ms/step - loss: 0.1632 - accuracy: 0.9442 - val_loss: 0.1635 - val_accuracy: 0.9439
Epoch 3/15
251/251 [==

In [18]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy= %.2f%%" % (scores[1]*100))

Accuracy= 93.21%
